# Import necessary packages

In [17]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [7]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Foods\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
len(sales_intermittent)

1301

In [9]:
len(sales_lumpy)

395

In [10]:
len(sales_erratic)

458

In [11]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [13]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1618,FOODS_1_008_CA_1_validation,FOODS_1_008,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1619,FOODS_1_009_CA_1_validation,FOODS_1_009,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,FOODS_1_010_CA_1_validation,FOODS_1_010,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
1621,FOODS_1_011_CA_1_validation,FOODS_1_011,FOODS_1,FOODS,CA_1,CA,2,1,0,0,...,0,2,0,1,3,2,3,1,0,1
1627,FOODS_1_017_CA_1_validation,FOODS_1_017,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,2,2,0,1,0,0,1,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,FOODS_3_806_CA_4_validation,FOODS_3_806,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
12189,FOODS_3_821_CA_4_validation,FOODS_3_821,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,1,1,0,3,0,0
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1


# User-defined functions to calculate Metrics

In [14]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [18]:
sales_pattern = sales_intermittent.copy()

In [19]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [20]:
dataframe_weekly

,wm_yr_wk,FOODS_1_005_CA_2,FOODS_1_005_CA_4,FOODS_1_008_CA_1,FOODS_1_008_CA_4,FOODS_1_009_CA_1,FOODS_1_009_CA_2,FOODS_1_009_CA_3,FOODS_1_009_CA_4,FOODS_1_010_CA_1,...,FOODS_3_821_CA_4,FOODS_3_823_CA_1,FOODS_3_823_CA_2,FOODS_3_823_CA_3,FOODS_3_823_CA_4,FOODS_3_824_CA_1,FOODS_3_824_CA_2,FOODS_3_824_CA_4,FOODS_3_825_CA_4,FOODS_3_826_CA_3
0,11101,36.0,4.0,0.0,1.0,4.0,6.0,6.0,5.0,0.0,...,0.0,0.0,24.0,18.0,2.0,2.0,3.0,1.0,6.0,0.0
1,11102,37.0,13.0,0.0,3.0,5.0,18.0,19.0,5.0,0.0,...,0.0,0.0,30.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0
2,11103,40.0,3.0,0.0,3.0,0.0,8.0,4.0,2.0,0.0,...,0.0,16.0,8.0,31.0,20.0,0.0,0.0,0.0,0.0,0.0
3,11104,30.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,3.0,20.0,7.0,3.0,1.0,0.0,3.0,0.0
4,11105,33.0,8.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,38.0,6.0,0.0,1.0,2.0,5.0,16.0,6.0,3.0,...,0.0,0.0,17.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0
269,11609,19.0,13.0,0.0,1.0,4.0,18.0,9.0,0.0,0.0,...,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
270,11610,29.0,7.0,0.0,0.0,10.0,3.0,13.0,0.0,2.0,...,0.0,0.0,17.0,0.0,1.0,13.0,7.0,0.0,3.0,0.0
271,11611,43.0,7.0,0.0,2.0,5.0,6.0,7.0,3.0,0.0,...,0.0,0.0,31.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0


In [21]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [22]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [23]:
len(list_pattern_py)

1301

In [24]:
best_combination = (0, 1, 3)

In [25]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_008_CA_1
Progressing: 0.0 %
Currently Running Product: FOODS_1_009_CA_1
Progressing: 0.08 %
Currently Running Product: FOODS_1_010_CA_1
Progressing: 0.15 %
Currently Running Product: FOODS_1_011_CA_1
Progressing: 0.23 %
Currently Running Product: FOODS_1_017_CA_1
Progressing: 0.31 %
Currently Running Product: FOODS_1_019_CA_1
Progressing: 0.38 %
Currently Running Product: FOODS_1_022_CA_1
Progressing: 0.46 %
Currently Running Product: FOODS_1_023_CA_1
Progressing: 0.54 %
Currently Running Product: FOODS_1_029_CA_1
Progressing: 0.61 %
Currently Running Product: FOODS_1_035_CA_1
Progressing: 0.69 %
Currently Running Product: FOODS_1_041_CA_1
Progressing: 0.77 %
Currently Running Product: FOODS_1_044_CA_1
Progressing: 0.85 %
Currently Running Product: FOODS_1_048_CA_1
Progressing: 0.92 %
Currently Running Product: FOODS_1_056_CA_1
Progressing: 1.0 %
Currently Running Product: FOODS_1_057_CA_1
Progressing: 1.08 %
Currently Running Product: FOODS_1_065_CA_

Currently Running Product: FOODS_2_243_CA_1
Progressing: 10.07 %
Currently Running Product: FOODS_2_246_CA_1
Progressing: 10.15 %
Currently Running Product: FOODS_2_249_CA_1
Progressing: 10.22 %
Currently Running Product: FOODS_2_252_CA_1
Progressing: 10.3 %
Currently Running Product: FOODS_2_254_CA_1
Progressing: 10.38 %
Currently Running Product: FOODS_2_256_CA_1
Progressing: 10.45 %
Currently Running Product: FOODS_2_259_CA_1
Progressing: 10.53 %
Currently Running Product: FOODS_2_260_CA_1
Progressing: 10.61 %
Currently Running Product: FOODS_2_261_CA_1
Progressing: 10.68 %
Currently Running Product: FOODS_2_263_CA_1
Progressing: 10.76 %
Currently Running Product: FOODS_2_264_CA_1
Progressing: 10.84 %
Currently Running Product: FOODS_2_270_CA_1
Progressing: 10.91 %
Currently Running Product: FOODS_2_275_CA_1
Progressing: 10.99 %
Currently Running Product: FOODS_2_281_CA_1
Progressing: 11.07 %
Currently Running Product: FOODS_2_284_CA_1
Progressing: 11.15 %
Currently Running Product:

Currently Running Product: FOODS_3_326_CA_1
Progressing: 19.83 %
Currently Running Product: FOODS_3_329_CA_1
Progressing: 19.91 %
Currently Running Product: FOODS_3_334_CA_1
Progressing: 19.98 %
Currently Running Product: FOODS_3_336_CA_1
Progressing: 20.06 %
Currently Running Product: FOODS_3_338_CA_1
Progressing: 20.14 %
Currently Running Product: FOODS_3_341_CA_1
Progressing: 20.22 %
Currently Running Product: FOODS_3_342_CA_1
Progressing: 20.29 %
Currently Running Product: FOODS_3_345_CA_1
Progressing: 20.37 %
Currently Running Product: FOODS_3_350_CA_1
Progressing: 20.45 %
Currently Running Product: FOODS_3_352_CA_1
Progressing: 20.52 %
Currently Running Product: FOODS_3_362_CA_1
Progressing: 20.6 %
Currently Running Product: FOODS_3_364_CA_1
Progressing: 20.68 %
Currently Running Product: FOODS_3_370_CA_1
Progressing: 20.75 %
Currently Running Product: FOODS_3_371_CA_1
Progressing: 20.83 %
Currently Running Product: FOODS_3_373_CA_1
Progressing: 20.91 %
Currently Running Product:

Currently Running Product: FOODS_1_102_CA_2
Progressing: 29.75 %
Currently Running Product: FOODS_1_104_CA_2
Progressing: 29.82 %
Currently Running Product: FOODS_1_105_CA_2
Progressing: 29.9 %
Currently Running Product: FOODS_1_109_CA_2
Progressing: 29.98 %
Currently Running Product: FOODS_1_116_CA_2
Progressing: 30.05 %
Currently Running Product: FOODS_1_117_CA_2
Progressing: 30.13 %
Currently Running Product: FOODS_1_119_CA_2
Progressing: 30.21 %
Currently Running Product: FOODS_1_123_CA_2
Progressing: 30.28 %
Currently Running Product: FOODS_1_124_CA_2
Progressing: 30.36 %
Currently Running Product: FOODS_1_125_CA_2
Progressing: 30.44 %
Currently Running Product: FOODS_1_126_CA_2
Progressing: 30.51 %
Currently Running Product: FOODS_1_135_CA_2
Progressing: 30.59 %
Currently Running Product: FOODS_1_138_CA_2
Progressing: 30.67 %
Currently Running Product: FOODS_1_146_CA_2
Progressing: 30.75 %
Currently Running Product: FOODS_1_156_CA_2
Progressing: 30.82 %
Currently Running Product:

Currently Running Product: FOODS_2_295_CA_2
Progressing: 39.74 %
Currently Running Product: FOODS_2_297_CA_2
Progressing: 39.82 %
Currently Running Product: FOODS_2_298_CA_2
Progressing: 39.89 %
Currently Running Product: FOODS_2_300_CA_2
Progressing: 39.97 %
Currently Running Product: FOODS_2_301_CA_2
Progressing: 40.05 %
Currently Running Product: FOODS_2_303_CA_2
Progressing: 40.12 %
Currently Running Product: FOODS_2_304_CA_2
Progressing: 40.2 %
Currently Running Product: FOODS_2_306_CA_2
Progressing: 40.28 %
Currently Running Product: FOODS_2_310_CA_2
Progressing: 40.35 %
Currently Running Product: FOODS_2_312_CA_2
Progressing: 40.43 %
Currently Running Product: FOODS_2_317_CA_2
Progressing: 40.51 %
Currently Running Product: FOODS_2_320_CA_2
Progressing: 40.58 %
Currently Running Product: FOODS_2_325_CA_2
Progressing: 40.66 %
Currently Running Product: FOODS_2_330_CA_2
Progressing: 40.74 %
Currently Running Product: FOODS_2_333_CA_2
Progressing: 40.81 %
Currently Running Product:

...

Currently Running Product: FOODS_2_005_CA_3
Progressing: 59.65 %
Currently Running Product: FOODS_2_006_CA_3
Progressing: 59.72 %
Currently Running Product: FOODS_2_015_CA_3
Progressing: 59.8 %
Currently Running Product: FOODS_2_032_CA_3
Progressing: 59.88 %
Currently Running Product: FOODS_2_046_CA_3
Progressing: 59.95 %
Currently Running Product: FOODS_2_068_CA_3
Progressing: 60.03 %
Currently Running Product: FOODS_2_087_CA_3
Progressing: 60.11 %
Currently Running Product: FOODS_2_090_CA_3
Progressing: 60.18 %
Currently Running Product: FOODS_2_117_CA_3
Progressing: 60.26 %
Currently Running Product: FOODS_2_123_CA_3
Progressing: 60.34 %
Currently Running Product: FOODS_2_127_CA_3
Progressing: 60.42 %
Currently Running Product: FOODS_2_129_CA_3
Progressing: 60.49 %
Currently Running Product: FOODS_2_154_CA_3
Progressing: 60.57 %
Currently Running Product: FOODS_2_158_CA_3
Progressing: 60.65 %
Currently Running Product: FOODS_2_159_CA_3
Progressing: 60.72 %
Currently Running Product:

Currently Running Product: FOODS_1_015_CA_4
Progressing: 69.56 %
Currently Running Product: FOODS_1_017_CA_4
Progressing: 69.64 %
Currently Running Product: FOODS_1_019_CA_4
Progressing: 69.72 %
Currently Running Product: FOODS_1_026_CA_4
Progressing: 69.79 %
Currently Running Product: FOODS_1_028_CA_4
Progressing: 69.87 %
Currently Running Product: FOODS_1_029_CA_4
Progressing: 69.95 %
Currently Running Product: FOODS_1_030_CA_4
Progressing: 70.02 %
Currently Running Product: FOODS_1_033_CA_4
Progressing: 70.1 %
Currently Running Product: FOODS_1_034_CA_4
Progressing: 70.18 %
Currently Running Product: FOODS_1_038_CA_4
Progressing: 70.25 %
Currently Running Product: FOODS_1_039_CA_4
Progressing: 70.33 %
Currently Running Product: FOODS_1_041_CA_4
Progressing: 70.41 %
Currently Running Product: FOODS_1_049_CA_4
Progressing: 70.48 %
Currently Running Product: FOODS_1_050_CA_4
Progressing: 70.56 %
Currently Running Product: FOODS_1_052_CA_4
Progressing: 70.64 %
Currently Running Product:

Currently Running Product: FOODS_2_123_CA_4
Progressing: 79.32 %
Currently Running Product: FOODS_2_126_CA_4
Progressing: 79.4 %
Currently Running Product: FOODS_2_127_CA_4
Progressing: 79.48 %
Currently Running Product: FOODS_2_128_CA_4
Progressing: 79.55 %
Currently Running Product: FOODS_2_129_CA_4
Progressing: 79.63 %
Currently Running Product: FOODS_2_140_CA_4
Progressing: 79.71 %
Currently Running Product: FOODS_2_145_CA_4
Progressing: 79.78 %
Currently Running Product: FOODS_2_149_CA_4
Progressing: 79.86 %
Currently Running Product: FOODS_2_154_CA_4
Progressing: 79.94 %
Currently Running Product: FOODS_2_165_CA_4
Progressing: 80.02 %
Currently Running Product: FOODS_2_166_CA_4
Progressing: 80.09 %
Currently Running Product: FOODS_2_170_CA_4
Progressing: 80.17 %
Currently Running Product: FOODS_2_184_CA_4
Progressing: 80.25 %
Currently Running Product: FOODS_2_186_CA_4
Progressing: 80.32 %
Currently Running Product: FOODS_2_190_CA_4
Progressing: 80.4 %
Currently Running Product: 

Currently Running Product: FOODS_3_206_CA_4
Progressing: 89.24 %
Currently Running Product: FOODS_3_211_CA_4
Progressing: 89.32 %
Currently Running Product: FOODS_3_219_CA_4
Progressing: 89.39 %
Currently Running Product: FOODS_3_220_CA_4
Progressing: 89.47 %
Currently Running Product: FOODS_3_233_CA_4
Progressing: 89.55 %
Currently Running Product: FOODS_3_235_CA_4
Progressing: 89.62 %
Currently Running Product: FOODS_3_236_CA_4
Progressing: 89.7 %
Currently Running Product: FOODS_3_249_CA_4
Progressing: 89.78 %
Currently Running Product: FOODS_3_256_CA_4
Progressing: 89.85 %
Currently Running Product: FOODS_3_260_CA_4
Progressing: 89.93 %
Currently Running Product: FOODS_3_261_CA_4
Progressing: 90.01 %
Currently Running Product: FOODS_3_264_CA_4
Progressing: 90.08 %
Currently Running Product: FOODS_3_281_CA_4
Progressing: 90.16 %
Currently Running Product: FOODS_3_284_CA_4
Progressing: 90.24 %
Currently Running Product: FOODS_3_285_CA_4
Progressing: 90.32 %
Currently Running Product:

Currently Running Product: FOODS_3_769_CA_4
Progressing: 99.0 %
Currently Running Product: FOODS_3_778_CA_4
Progressing: 99.08 %
Currently Running Product: FOODS_3_784_CA_4
Progressing: 99.15 %
Currently Running Product: FOODS_3_790_CA_4
Progressing: 99.23 %
Currently Running Product: FOODS_3_797_CA_4
Progressing: 99.31 %
Currently Running Product: FOODS_3_798_CA_4
Progressing: 99.39 %
Currently Running Product: FOODS_3_799_CA_4
Progressing: 99.46 %
Currently Running Product: FOODS_3_803_CA_4
Progressing: 99.54 %
Currently Running Product: FOODS_3_806_CA_4
Progressing: 99.62 %
Currently Running Product: FOODS_3_821_CA_4
Progressing: 99.69 %
Currently Running Product: FOODS_3_823_CA_4
Progressing: 99.77 %
Currently Running Product: FOODS_3_824_CA_4
Progressing: 99.85 %
Currently Running Product: FOODS_3_825_CA_4
Progressing: 99.92 %
This line of code took 1.6546736049999997 minutes


# Calculate metrics for optimal combination

In [26]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.040358123333333155 minutes


In [27]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_005_CA_2,0.493613,31.8950,32.6079,29.8532,10.365865,11.591897
1,FOODS_1_005_CA_4,1.161464,51.6206,66.2510,47.3818,4.645858,5.355637
2,FOODS_1_008_CA_1,inf,inf,200.0000,NaN,0.007065,0.007067
3,FOODS_1_008_CA_4,0.859341,73.6578,104.9092,66.9976,2.578023,3.007529
4,FOODS_1_009_CA_1,0.702009,23.4003,23.9915,22.7393,1.404017,1.479130
...,...,...,...,...,...,...,...
1296,FOODS_3_824_CA_1,0.800118,160.0235,106.0441,12.8658,1.600235,2.089062
1297,FOODS_3_824_CA_2,1.067691,213.5383,109.7575,17.7800,1.067691,1.389918
1298,FOODS_3_824_CA_4,0.528365,105.6730,123.3913,37.9141,0.528365,0.549033
1299,FOODS_3_825_CA_4,inf,321.9006,123.3306,321.9006,3.219006,3.220466


In [28]:
df_arima_metrics.MAPE.mean()

71.59245574712644

In [29]:
df_arima_metrics.RMSE.mean()

3.514190999113786

In [30]:
df_arima_metrics.to_csv('ARIMA_Intermittent_Forecast_Metrics.csv')